In [ ]:
import os
import sys
import glob

In [ ]:
import numpy as np
import scipy
import matplotlib.dates
import matplotlib.pyplot as plt
import datetime
import dateutil
from dateutil import rrule

In [ ]:
package_root = os.path.abspath('../../..')
if package_root not in sys.path:
    sys.path.append(package_root)
from seismic.ASDFdatabase import FederatedASDFDataSet

In [ ]:
import obspy
from seismic.xcorqc.analytic_plot_utils import distance
from netCDF4 import Dataset as NCDataset
import pandas as pd

In [ ]:
# Imports for plotting
from textwrap import wrap
from scipy import signal
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
from tqdm.auto import tqdm

In [ ]:
from seismic.xcorqc.xcorr_station_clock_analysis import plot_xcorr_file_clock_analysis

In [ ]:
ds = FederatedASDFDataSet.FederatedASDFDataSet("/g/data/ha3/Passive/SHARED_DATA/Index/asdf_files.txt")

In [ ]:
def read_correlator_config(nc_file):
    '''Read the correlator settings used for given nc file.
    '''
    folder, fname = os.path.split(nc_file)
    base, _ = os.path.splitext(fname)
    base_parts = base.split('.')
    if len(base_parts) > 4:
        timestamp = '.'.join(base_parts[4:7])
        config_filename = '.'.join(['correlator', timestamp, 'cfg'])
        config_file = os.path.join(folder, config_filename)
        if os.path.exists(config_file):
            settings_df = pd.read_csv(config_file, sep=':', names=['setting', 'value'],
                                      index_col=0, skiprows=1, skipinitialspace=True,
                                      squeeze=True, converters={'setting': str.strip})
            title_tag = '({}-{} Hz)'.format(settings_df['--fmin'], settings_df['--fmax'])
        else:
            print('WARNING: Settings file {} not found!'.format(config_file))
            settings_df = None
            title_tag = ''
    else:
        settings_df = None
        title_tag = ''

    return settings_df, title_tag

In [ ]:
TIME_WINDOW = 600 # +/-
# TIME_WINDOW = 300 # +/-
# TIME_WINDOW = 120 # +/-
# SNR_THRESHOLD = 10
SNR_THRESHOLD = 6

In [ ]:
NCFILE_PATH = '/g/data/ha3/am7399/shared/xcorr/AU/QIS_Warramunga'
# NCFILE_PATH = '/home/547/am7399/am7399/shared/xcorr/7X/MAx_MAy'

SRC_FILES = glob.glob(os.path.join(NCFILE_PATH, '*.nc'))

print("Found {} .nc files".format(len(SRC_FILES)))

In [ ]:
import gc

save_plots = True
underlay_rcf_xcorr = False

pbar = tqdm(total=len(SRC_FILES))
found_preexisting = False
failed_files = []
for src_file in SRC_FILES:
    _, base_file = os.path.split(src_file)
    pbar.set_description(base_file)
    pbar.update()
    # Extract timestamp from nc filename if available
    settings, title_tag = read_correlator_config(src_file)
    
    assert os.path.exists(src_file), "File {} not found!".format(src_file)

    try:
        if save_plots:
            basename, _ = os.path.splitext(src_file)
            png_file = basename + ".png"
            # If png file already exists and has later timestamp than src_file, then skip it.
            if os.path.exists(png_file):
                src_file_time = os.path.getmtime(src_file)
                png_file_time = os.path.getmtime(png_file)
                if png_file_time > src_file_time:
                    tqdm.write("PNG file {} is more recent than source file {}, skipping!".format(
                               os.path.split(png_file)[1], os.path.split(src_file)[1]))
                    found_preexisting = True
                    continue
            plot_xcorr_file_clock_analysis(src_file, ds, TIME_WINDOW, SNR_THRESHOLD, png_file=png_file,
                                           show=False, underlay_rcf_xcorr=underlay_rcf_xcorr,
                                           title_tag=title_tag, settings=settings)
        else:
            plot_xcorr_file_clock_analysis(src_file, ds, TIME_WINDOW, SNR_THRESHOLD, 
                                           underlay_rcf_xcorr=underlay_rcf_xcorr, title_tag=title_tag,
                                           settings=settings)
        if gc.garbage:
            tqdm.write("WARNING: {} items of uncollectable garbage".format(len(gc.garbage)))

    except Exception as e:
        tqdm.write("ERROR processing file {}".format(src_file))
        failed_files.append((src_file, e.message))
        
pbar.close()

if failed_files:
    print("The following files experienced errors:")
    for f, e in failed_files:
        print(" File: " + f)
        print("Error: " + e)

if found_preexisting:
    print("Some files were skipped because pre-existing matching png files were up-to-date.\n"
          "Remove png files to force regeneration.")